# Linegraph - ipyrun Tutorial 

In [1]:
import sys
import pathlib
import subprocess
from pydantic import validator

from ipyautoui.autodisplayfile_renderers import display_python_file
from ipyautoui import AutoUi
from ipyautoui import AutoDisplay

sys.path.append(str(pathlib.Path('../../../src').resolve())) # append ipyrun. TODO: for dev only. remove at build time.

from ipyrun import RunApp, BatchApp, ConfigShell, DefaultConfigShell, RunShellActions 
from ipyrun.runshell import AutoDisplayDefinition, create_autodisplay_map, ConfigBatch, DefaultConfigShell, DefaultBatchActions, BatchShellActions

In [2]:
#  for supporting info see: 
#  ?RunApp
#  ?DefaultConfigShell
#  ?RunShellActions

In [3]:
#?RunApp
#?DefaultConfigShell

## Define Brief for App

- given a set of inputs, create a line graph of a defined range
- __note__. `ipyrun` will deal with the caching of the data





## Create a working directory 

In [4]:
from seedir import seedir

DIR_CORE = pathlib.Path('./linegraph_core').resolve()
PATH_RUN = DIR_CORE
FPTH_INPUTS_SCHEMA = DIR_CORE / 'input_schema_linegraph.py'
DIR_APPDATA = pathlib.Path('./linegraph_appdata').resolve()
DIR_APPDATA.mkdir(exist_ok=True, parents=True)

seedir(exclude_folders=['.ipynb_checkpoints', '__pycache__'], style='emoji')

📁 linegraph/
├─📄 linegraph_app.ipynb
├─📁 linegraph_core/
│ ├─📄 __main__.py
│ ├─📄 script_linegraph.py
│ ├─📄 make_graph.py
│ ├─📄 input_schema_linegraph.py
│ └─📄 __init__.py
├─📁 linegraph_appdata/
├─📄 linegraph_app_tutorial.ipynb
└─📄 __init__.py


## App Inputs

### Create pydantic model schema for App inputs

- what are the required input fields required for plotting a line graph? 
- create a pydantic schema of these inputs

In [5]:
from linegraph_core.input_schema_linegraph import LineGraph
display_python_file(FPTH_INPUTS_SCHEMA)  # it is advised that input_schema's get their own python file


```python

from pydantic import BaseModel , Field, conint

#sys.path.append(pathlib.Path(__file__).parents[3])

class LineGraph(BaseModel):
    """parameters to define a simple `y=m*x + c` line graph"""
    title: str = Field(default='line equation', description='add chart title here')
    m: float = Field(default=2, description='gradient')
    c: float = Field(default=5, description='intercept')
    x_range: tuple[int, int] = Field(default=(0,5), ge=0, le=50, description='x-range for chart')
    y_range: tuple[int, int] = Field(default=(0,5), ge=0, le=50, description='y-range for chart')
        
if __name__ == "__main__":
    lg = LineGraph()
```


### Create an inputs UI from model with AutoUi

- check the UI looks correct based on the pydantic model
- save the test file (`test.lg.json`) so we can test loading it back in


In [6]:
?AutoUi

Init signature:
AutoUi(
    schema: Union[Type[pydantic.main.BaseModel], dict],
    value: dict = None,
    path: pathlib.Path = None,
    save_controls: ipyautoui.autoui.SaveControls = <SaveControls.save_buttonbar: 'save_buttonbar'>,
    show_raw: bool = True,
    fn_onsave: Union[Callable, List[Callable]] = <function AutoUi.<lambda> at 0x7f04b8b3a310>,
    validate_onchange=True,
    update_fdir_to_path_parent=True,
)
Docstring:     
extends AutoIpywidget and AutoUiCommonMethods to create an 
AutoUi capable of interacting with a json file
Init docstring: Public constructor
File:           ~/miniconda3/envs/ipyrun-dev/lib/python3.9/site-packages/ipyautoui/autoui.py
Type:           MetaHasTraits
Subclasses:     


In [7]:
lg = LineGraph()
AutoUi(schema=LineGraph, value=lg.dict(), path='in-test.lg.json')

AutoUi(children=(VBox(children=(HBox(children=(SaveButtonBar(children=(ToggleButton(value=False, button_style=…

In [8]:
from ipyrun.basemodel import file
setattr(LineGraph, 'file', file)
LineGraph().file('in-test.lg.json')

### Create a AutoDisplay renderer, and check it works

- extend the default AutoDisplay renderer to be able to render `*.lg.json` files


In [9]:
# ?AutoDisplayDefinition
lg_inputs_dfd = AutoDisplayDefinition(
    path=FPTH_INPUTS_SCHEMA,
    obj_name='LineGraph',
    ftype='in',
    ext='.lg.json'
)

# This is purely for illustration of the renderer. This is done within the app for you.
lg_inputs_renderer = AutoUi.create_autodisplay_map(schema=LineGraph, ext='.lg.json')
AutoDisplay.from_paths(paths=['in-test.lg.json'],file_renderers=lg_inputs_renderer)

In [10]:
lg_inputs_dfd.dict()

{'path': PosixPath('/home/jovyan/git_mf/ipyrun/docs/examples/linegraph/linegraph_core/input_schema_linegraph.py'),
 'obj_name': 'LineGraph',
 'module_name': 'input_schema_linegraph',
 'ftype': <FiletypeEnum.input: 'in'>,
 'ext': '.lg.json'}

In [11]:
lg_inputs_renderer

{'.lg.json': ipyautoui.autoui.AutoUiCommonMethods.create_autoui_renderer.<locals>.AutoRenderer}

## Write main App code

- This is where you put the main code that fulfils the original brief
- We are using the `RunShell` App so the main code lives in a script that is executed on the command line
- There must be a way to pass input variables to this script
    - The default way in `RunShell` is that the filepath of the inputs and outputs files are passed as a `sys.argv`s
- In the simple example below, the script can be tested by simply running:
    - `python -O -m linegraph_core in-test.lg.json out-test.csv out-test.plotly.json`

In [12]:
subprocess.call('python -O -m linegraph_core in-test.lg.json out-test.csv out-test.plotly.json', shell=True)

fpth_in = in-test.lg.json
fpth_out_csv = out-test.csv
fpth_out_plotly = out-test.plotly.json


0

## Create a RunApp

### Set-up App Config

- Now we have an inputs file, and a script that can be executed to create the desired outputs
- We must now configure the App which will handle the caching of input and output data
    - This allows the Notebook to be run as an App, such that a single Notebook can be run by any job, with the job data in the job 

In [13]:
#?DefaultConfigShell

In [14]:
lg_inputs_dfd.dict()

{'path': PosixPath('/home/jovyan/git_mf/ipyrun/docs/examples/linegraph/linegraph_core/input_schema_linegraph.py'),
 'obj_name': 'LineGraph',
 'module_name': 'input_schema_linegraph',
 'ftype': <FiletypeEnum.input: 'in'>,
 'ext': '.lg.json'}

- As you can see above, given this set of inputs the shell command has been generated 
- We can either: 
    - 1. explictly give fpths_outputs as a variable, or 
    - 2. extend `DefaultConfigShell` to generate `fpths_outputs` from other variables
- option 2. is favourable as it can then be easily extended to an arbitrary number of runs, and the names of the output files will follow the same logical naming structure as the input files

In [15]:
# DefaultConfigShell defines how to cache data to be run by the App

class LineGraphConfigShell(DefaultConfigShell):    
    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values["fdir_appdata"]
        key = values["key"]
        paths = [
            fdir / ("out-" + key + ".csv"),
            fdir / ("out-" + key + ".plotly.json"),
        ]
        return paths
    
config = LineGraphConfigShell(
    path_run=PATH_RUN,
    fdir_appdata=DIR_APPDATA,
    autodisplay_definitions=[lg_inputs_dfd],
)
display(config.dict())

{'index': 0,
 'path_run': PosixPath('/home/jovyan/git_mf/ipyrun/docs/examples/linegraph/linegraph_core'),
 'pythonpath': PosixPath('/home/jovyan/git_mf/ipyrun/docs/examples/linegraph'),
 'run': 'linegraph_core',
 'name': 'linegraph_core',
 'long_name': '00 - Linegraph Core',
 'key': '00-linegraph_core',
 'fdir_root': PosixPath('.'),
 'fdir_appdata': PosixPath('00-linegraph_core'),
 'in_batch': False,
 'status': None,
 'update_config_at_runtime': False,
 'autodisplay_definitions': [{'path': PosixPath('/home/jovyan/git_mf/ipyrun/docs/examples/linegraph/linegraph_core/input_schema_linegraph.py'),
   'obj_name': 'LineGraph',
   'module_name': 'input_schema_linegraph',
   'ftype': <FiletypeEnum.input: 'in'>,
   'ext': '.lg.json'}],
 'autodisplay_inputs_kwargs': {},
 'autodisplay_outputs_kwargs': {},
 'fpths_inputs': [PosixPath('00-linegraph_core/in-00-linegraph_core.lg.json')],
 'fpths_outputs': [PosixPath('00-linegraph_core/out-00-linegraph_core.csv'),
  PosixPath('00-linegraph_core/out-00

### Build the RunApp

In [16]:
#  RunApp is described as UI first -
#  as such it can be loaded and viewed before all of the buttons have been assigned useful callable functions 
#  pressing the buttons below will yield simple default outputs

RunApp(config=config, cls_actions=RunShellActions)

RunApp(children=(HBox(children=(Checkbox(value=False, indent=False, layout=Layout(height='40px', max_width='20…

## Create a BatchApp

- BatchApp's support an arbitrary number of runs 
- This extends the app to be able to add and remove runs to suit the users requirements
- To be able to do this, we need to be able to build a new RunApp on a button click
    - this requires `LineGraphConfigShell` to be fully defined apart from a single variable (the `index` that will be passed from the BatchApp) 

In [17]:

class LineGraphConfigShell(DefaultConfigShell):
    @validator("path_run", always=True, pre=True)
    def _set_path_run(cls, v, values):
        return PATH_RUN

    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values["fdir_appdata"]
        key = values["key"]
        paths = [
            fdir / ("out-" + key + ".csv"),
            fdir / ("out-" + key + ".plotly.json"),
        ]
        return paths

    @validator("autodisplay_definitions", always=True)
    def _autodisplay_definitions(cls, v, values):
        return [
            AutoDisplayDefinition(
                path=FPTH_INPUTS_SCHEMA,
                obj_name="LineGraph",
                ext=".lg.json",
                ftype='in',
            )
        ]

    @validator("autodisplay_inputs_kwargs", always=True)
    def _autodisplay_inputs_kwargs(cls, v, values):
        return dict(patterns="*")

    @validator("autodisplay_outputs_kwargs", always=True)
    def _autodisplay_outputs_kwargs(cls, v, values):
        return dict(patterns="*.plotly.json")

config = LineGraphConfigShell(fdir_appdata=DIR_APPDATA)
run_app = RunApp(config, cls_actions=RunShellActions)  # cls_ui=RunUi,
display(run_app)

RunApp(children=(HBox(children=(Checkbox(value=False, indent=False, layout=Layout(height='40px', max_width='20…

In [18]:
config_batch = ConfigBatch(
    fdir_root=DIR_APPDATA,
    cls_config=LineGraphConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)

display(config_batch.dict())
print('---')

{'fdir_root': PosixPath('/home/jovyan/git_mf/ipyrun/docs/examples/linegraph/linegraph_appdata'),
 'fpth_config': PosixPath('config-shell_handler.json'),
 'title': '# Plot Straight Lines\n### example RunApp',
 'status': None,
 'configs': []}

---


### Extend Batch Actions

In [19]:
class LineGraphConfigBatch(ConfigBatch):

    @validator("cls_config", always=True)
    def _cls_config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        return LineGraphConfigShell

class LineGraphBatchActions(BatchShellActions):
    @validator("config", always=True)
    def _config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        if type(v) == dict:
            v = LineGraphConfigBatch(**v)
        return v

    @validator("runlog_show", always=True)
    def _runlog_show(cls, v, values):
        return None

config_batch = LineGraphConfigBatch(
    fdir_root=DIR_APPDATA,
    # cls_config=MyConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)
if config_batch.fpth_config.is_file():
    config_batch = LineGraphConfigBatch.parse_file(config_batch.fpth_config)
app = BatchApp(config_batch, cls_actions=LineGraphBatchActions)
display(app)

BatchApp(children=(VBox(children=(HTML(value='<h1>Plot Straight Lines</h1>\n<h3>example RunApp</h3>'), HBox(ch…

### Customise the BatchApp Interface

#### Editable Names for each RunApp

In [20]:
from ipyrun.ui_add import AddNamedRun
import functools
from IPython.display import clear_output
import stringcase

In [21]:
def fn_add_show(app):
    clear_output()
    display(AddNamedRun(fn_add=app.actions.add))


class LineGraphConfigBatch(ConfigBatch):
    
    @validator("cls_config", always=True)
    def _cls_config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        return LineGraphConfigShell

class LineGraphBatchActions(BatchShellActions):
    
    @validator("add_show", always=True)
    def _add_show(cls, v, values):
        return functools.partial(fn_add_show, values["app"])
    
    @validator("config", always=True)
    def _config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        if type(v) == dict:
            v = LineGraphConfigBatch(**v)
        return v

    @validator("runlog_show", always=True)
    def _runlog_show(cls, v, values):
        return None

config_batch = LineGraphConfigBatch(
    fdir_root=DIR_APPDATA,
    # cls_config=MyConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)
if config_batch.fpth_config.is_file():
    config_batch = LineGraphConfigBatch.parse_file(config_batch.fpth_config)
app = BatchApp(config_batch, cls_actions=LineGraphBatchActions)
display(app)

BatchApp(children=(VBox(children=(HTML(value='<h1>Plot Straight Lines</h1>\n<h3>example RunApp</h3>'), HBox(ch…

#### Drop the numbers from the "long_name" of the run, and from the "key" is used as the filename

```{note}
by removing the index from the key you are ensuring unique names
```


In [22]:
class NewLineGraphConfigShell(DefaultConfigShell):
    @validator("path_run", always=True, pre=True)
    def _set_path_run(cls, v, values):
        return PATH_RUN

    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values["fdir_appdata"]
        key = values["key"]
        paths = [
            fdir / ("out-" + key + ".csv"),
            fdir / ("out-" + key + ".plotly.json"),
        ]
        return paths

    @validator("autodisplay_definitions", always=True)
    def _autodisplay_definitions(cls, v, values):
        return [
            AutoDisplayDefinition(
                path=FPTH_INPUTS_SCHEMA,
                obj_name="LineGraph",
                ext=".lg.json",
                ftype='in',
            )
        ]

    @validator("autodisplay_inputs_kwargs", always=True)
    def _autodisplay_inputs_kwargs(cls, v, values):
        return dict(patterns="*")

    @validator("autodisplay_outputs_kwargs", always=True)
    def _autodisplay_outputs_kwargs(cls, v, values):
        return dict(patterns="*.plotly.json")
    
    #####################################################
    @validator("long_name", always=True)
    def _long_name(cls, v, values):
        if v is None:
            return stringcase.titlecase(values["name"])
        else:
            return v

    @validator("key", always=True)
    def _key(cls, v, values):
        if v is None:
            return values["name"]
        else:
            return v
    # ^ ADDED THIS BIT ###################################
    
    
def fn_add_show(app):
    clear_output()
    display(AddNamedRun(fn_add=app.actions.add))


class LineGraphConfigBatch(ConfigBatch):
    
    @validator("cls_config", always=True)
    def _cls_config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        return NewLineGraphConfigShell

class LineGraphBatchActions(BatchShellActions):
    
    @validator("add_show", always=True)
    def _add_show(cls, v, values):
        return functools.partial(fn_add_show, values["app"])
    
    @validator("config", always=True)
    def _config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        if type(v) == dict:
            v = LineGraphConfigBatch(**v)
        return v

    @validator("runlog_show", always=True)
    def _runlog_show(cls, v, values):
        return None

config_batch = LineGraphConfigBatch(
    fdir_root=DIR_APPDATA,
    # cls_config=MyConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)
if config_batch.fpth_config.is_file():
    config_batch = LineGraphConfigBatch.parse_file(config_batch.fpth_config)
app = BatchApp(config_batch, cls_actions=LineGraphBatchActions)
display(app)

BatchApp(children=(VBox(children=(HTML(value='<h1>Plot Straight Lines</h1>\n<h3>example RunApp</h3>'), HBox(ch…

#### Edit default show/hide settings upon activating panel


In [23]:
def show(app):
    print('show')
    app.help_ui.value = False
    app.help_run.value = False
    app.help_config.value = False
    app.inputs.value = True
    app.outputs.value = False
    app.runlog.value = True
    
    
class LineGraphActions(RunShellActions):
    @validator("activate", always=True)
    def _activate(cls, v, values):
        return functools.partial(show, values['app'])
    
config_batch = LineGraphConfigBatch(
    fdir_root=DIR_APPDATA,
    # cls_config=MyConfigShell,
    title="""# Plot Straight Lines\n### example RunApp""",
)
if config_batch.fpth_config.is_file():
    config_batch = LineGraphConfigBatch.parse_file(config_batch.fpth_config)
app = BatchApp(config_batch, cls_actions=LineGraphBatchActions)
display(app)

BatchApp(children=(VBox(children=(HTML(value='<h1>Plot Straight Lines</h1>\n<h3>example RunApp</h3>'), HBox(ch…